<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2024Project/blob/Szablon-wczytywania-danych-w-sparku/colabs/Wczytywanie_danych_NASA_w_sparku.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wczytywanie danych w sparku

Utworzenie środowiska pyspark do obliczeń:

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q dlcdn.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

In [ ]:
import findspark
findspark.init()

In [ ]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, FloatType, StringType, StructType
from google.colab import drive

Utowrzenie sesji:

In [2]:
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

Połączenie z dyskiem:

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


Wczytanie danych NASA znajdujących się na dysku w sparku:

In [27]:
columns = ['lon', 'lat', 'Date', 'SWdown', 'LWdown', 'SWnet', 'LWnet', 'Qle', 'Qh', 'Qg', 'Qf', 'Snowf', 'Rainf', 'Evap', 'Qs', 'Qsb', 'Qsm', 'AvgSurfT', 'Albedo', 'SWE', 'SnowDepth', 'SnowFrac', 'SoilT_0_10cm', 'SoilT_10_40cm', 
           'SoilT_40_100cm', 'SoilT_100_200cm', 'SoilM_0_10cm', 'SoilM_10_40cm', 'SoilM_40_100cm', 'SoilM_100_200cm', 'SoilM_0_100cm', 'SoilM_0_200cm', 'RootMoist', 'SMLiq_0_10cm', 'SMLiq_10_40cm', 'SMLiq_40_100cm', 'SMLiq_100_200cm', 
           'SMAvail_0_100cm', 'SMAvail_0_200cm', 'PotEvap', 'ECanop', 'TVeg', 'ESoil', 'SubSnow', 'CanopInt', 'ACond', 'CCond', 'RCS', 'RCT', 'RCQ', 'RCSOL', 'RSmin','RSMacr', 'LAI', 'GVEG', 'Streamflow']

# Utworzenie schematu określającego typ zmiennych
schema = StructType()
for i in columns:
  if i == "Date":
    schema = schema.add(i, StringType(), True)
  else:
    schema = schema.add(i, FloatType(), True)

In [ ]:
# Wczytanie zbioru Nasa w sparku
nasa = spark.read.format('csv').option("header", True).schema(schema).load('/content/drive/MyDrive/BigMess/NASA/NASA.csv')

Zanim zaczniemy pisać kwerendy należy jeszcze dodać nasz DataFrame (df) do "przestrzeni nazw tabel" Sparka:

In [29]:
nasa.createOrReplaceTempView("nasa")

Rodzielenie kolumny "Date" na kolumny "Year" oraz "Month"

In [34]:
nasa = spark.sql("""
          SELECT
          CAST(SUBSTRING(CAST(Date AS STRING), 1, 4) AS INT) AS Year,
          CAST(SUBSTRING(CAST(Date AS STRING), 5, 2) AS INT) AS Month,
          n.*
          FROM nasa n
          """)

In [37]:
nasa = nasa.drop("Date")
nasa.show(5)

+----+-----+---------+-------+---------+---------+---------+---------+---------+---------+----------+---+-----+-----+---------+---+----------+---+---------+---------+---+---------+--------+------------+-------------+--------------+---------------+------------+-------------+--------------+---------------+-------------+-------------+----------+------------+-------------+--------------+---------------+---------------+---------------+---------+----------+---------+----------+-------+------------+-----------+------------+----------+----------+----------+----------+---------+------------+---------+----------+----------+
|Year|Month|      lon|    lat|   SWdown|   LWdown|    SWnet|    LWnet|      Qle|       Qh|        Qg| Qf|Snowf|Rainf|     Evap| Qs|       Qsb|Qsm| AvgSurfT|   Albedo|SWE|SnowDepth|SnowFrac|SoilT_0_10cm|SoilT_10_40cm|SoilT_40_100cm|SoilT_100_200cm|SoilM_0_10cm|SoilM_10_40cm|SoilM_40_100cm|SoilM_100_200cm|SoilM_0_100cm|SoilM_0_200cm| RootMoist|SMLiq_0_10cm|SMLiq_10_40cm|SMLiq_4